In [14]:
from tensorflow.keras.models import load_model
import os

%load_ext autoreload 
%autoreload 2

# Load the trained encoder
encoder = load_model("../gesture_encoder_model.keras")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
from raw_to_dataset import load_gesture_xyz
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

def get_distance(filename1, filename2):
    gesture1 = load_gesture_xyz(filename1, use_centroid=True)
    gesture2 = load_gesture_xyz(filename2, use_centroid=True)
    # Pad sequences w/ zeros
    gestures_padded = pad_sequences([gesture1, gesture2], padding='post', dtype='float32')

    gest1_padded = gestures_padded[0:1]  # shape (1, seq_len, 3)
    gest2_padded = gestures_padded[1:2]

    emb1 = encoder.predict(gest1_padded)
    emb2 = encoder.predict(gest2_padded)

    # L2 distance between embeddings
    distance = np.linalg.norm(emb1 - emb2)
    return distance




In [16]:
# gest_filepath = "sample_data/square_eval/square_eval1_big.json"
gest_filepath = "sample_data/new_shape_spiral/new_shape_spiral_default.json"
# gest_filepath = "sample_data/circle_eval/circle_eval2.json"

square_dist = get_distance(gest_filepath, "sample_data/square_eval/square_eval2.json")
spiral_dist = get_distance(gest_filepath, "sample_data/new_shape_spiral/new_shape_spiral_default2.json")
circle_dist = get_distance(gest_filepath, "sample_data/circle_eval/circle_eval1.json")

print(f"Square dist: {square_dist:.4f}")
print(f"Spiral dist: {spiral_dist:.4f}")
print(f"Circle dist: {circle_dist:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 319ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 307ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Square dist: 0.3167
Spiral dist: 0.7798
Circle dist: 1.1294


In [19]:
base_dir = "sample_data"
# reference_filepath = "sample_data/circle_eval/circle_eval1.json"
# reference_filepath = "sample_data/square_eval/square_eval1_big.json"
reference_filepath = "sample_data/new_shape_spiral/new_shape_spiral_default2.json"

# Dictionary to store average distance for each gesture
avg_distances = {}

# Find average of comparisons to each gesture class' examples
for folder in os.listdir(base_dir):
    folder_path = os.path.join(base_dir, folder)

    if not os.path.isdir(folder_path):
        continue

    distances = []

    # Get score of each gesture
    for filename in os.listdir(folder_path):
        if not filename.endswith(".json"):
            continue

        gesture_path = os.path.join(folder_path, filename)

        # Skip the reference file itself in case using sample data as example
        if gesture_path == reference_filepath:
            continue

        dist = get_distance(reference_filepath, gesture_path)
        distances.append(dist)

    if distances:
        avg_distances[folder] = sum(distances) / len(distances)
    else:
        avg_distances[folder] = None  # no valid files

# Print results
for folder, avg in avg_distances.items():
    if avg is None:
        print(f"{folder}: (no gestures found)")
    else:
        print(f"{folder}: {avg:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
circle_eval: 1.4284
new_shape_spiral: 0.7477
square_eval: 0.9451
